In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

# Util built for sharktank project
from sharktank_utils import load_facts, OrchestratorResponse, PitchOrchestrator, metrics_calculation

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [ ]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [5]:
facts_store.keys()

dict_keys(['facts_shark_tank_transcript_12_Ornament Anchor.txt', 'facts_shark_tank_transcript_13_MAXPRO SmartConnect.txt', 'facts_shark_tank_transcript_32_Crispy Cones.txt', 'facts_shark_tank_transcript_33_Happi Floss.txt'])

In [6]:
if __name__ == "__main__":
    error_count = 0
    round_count = 0
    orchestrator = PitchOrchestrator()
    goal = 'create a pitch with the given facts'
    for case, facts in tqdm(facts_store.items()):
    # case = 'facts_shark_tank_transcript_0_GarmaGuard.txt'
        # if round_count == 1:
        #     break
        if pitches[case] != "":
            orchestrator.logs = []
            orchestrator.agents = {}
            round_count += 1
            continue
        else:
            lean_facts = {}
            lean_facts['facts'], lean_facts['product_description'] = facts['facts'], facts['product_description']
            try:
                try:
                    # Generate sub task
                    subtasks = orchestrator.generate_subtasks(goal, lean_facts)
                except Exception as e:
                    print('error at substasks')
                    print(e, "\nsubtasks:", subtasks)
                try:
                    orchestrator.create_agents(subtasks, lean_facts)
                except Exception as e:
                    print('error at agent creation')
                    print(e, "\nagent creation issue")
                try:
                    agent_outputs = orchestrator.run_agents(subtasks)
                except Exception as e:
                    print('error at agent outputs')
                    print(e, "\nagent outputs:", agent_outputs)
                try:
                    pitch = orchestrator.synthesize_pitch(agent_outputs)
                    pitches[case] = pitch
                    metrics[case] = orchestrator.logs
                    time_stamps[case] = datetime.now().strftime("%Y-%m-%d $H:%M:%S")
                except Exception as e:
                    error_count += 1
                    print('error at pitching')
                    print(e, "\pitch:", pitch)
                    pitch = ""
            except Exception as e:
                print(e, "case:", case)
                pitches[case] = ""
            # Reset agents
            orchestrator.logs = []
            orchestrator.agents = {}
            round_count += 1


 25%|██▌       | 1/4 [00:14<00:42, 14.15s/it]

ERROR    Error calling Groq API: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been  
         decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for 
         a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code':                
         'model_decommissioned'}}

WARNING  Attempt 1/1 failed: {"error":{"message":"The model `mixtral-8x7b-32768` has been decommissioned and is no 
         longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which
         model to use instead.","type":"invalid_request_error","code":"model_decommissioned"}}

ERROR    Failed after 1 attempts. Last error using Groq(mixtral-8x7b-32768)

error at agent outputs
{"error":{"message":"The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.","type":"invalid_request_error","code":"model_decommissioned"}}
 
agent outputs: {'Organize Key Points': "\n<think>\nOkay, let's start by looking at the user's query. They want me to extract and organize relevant information from the given facts, focusing on sales data, product features, and target audience. \n\nFirst, I'll need to parse the facts provided. The sales data includes the first year's sales during the pandemic at $312,000, June's summer month sales at $80,000, and the lifetime total over half a million in 18 months. I should note those numbers exactly as given.\n\nNext, the product features are listed under the product description. The Ornament Anchor can hold up to 10 pounds, prevents pets from knocking off ornaments, and is easy to use. 

100%|██████████| 4/4 [00:43<00:00, 10.95s/it]


In [7]:
length = 0
for k, v in pitches.items():
    if v == "":
        print(k)
        length += 1
print(length)

0


In [10]:
calculated_metrics = []
for product, metric in metrics.items():
    calculated_metrics.append(metrics_calculation(metric))
input_lengths, output_lengths, latencies = zip(*calculated_metrics)

In [11]:
timestamp = datetime.now().strftime('%Y-%m-%d')
framework = "orchestrator"
layer = "orchestrator_1"
pitches_df = pd.DataFrame(
    data={
        'scenario_name':list(pitches.keys()),
        'framework':len(pitches) * [framework],
        'layer':len(pitches) * [layer],
        'model_name': len(pitches) * ["groq/" + orchestrator.orchestrator],
        'model_identity':len(pitches) * ["groq/" + orchestrator.orchestrator],
        'latency':latencies,
        'input_length':input_lengths,
        'output_length':output_lengths,
        'time_stamp':list(time_stamps.values()),
        'response':list(pitches.values()),
    }
)


In [12]:
pitches_df.to_excel(f"{timestamp}-orchestrator-basic-pitches-03.xlsx")